<a href="https://colab.research.google.com/github/FiddiCoder/-FIA-Project/blob/main/ml_pipeline/Movie_Recommender_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm_notebook as tqdm
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt

In [ ]:
import os
import tarfile
from six.moves import urllib

#download dataset

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/FiddiCoder/-FIA-Project/main/"
FILE_PATH = os.path.join("dataset")
FILE_URL1 = DOWNLOAD_ROOT + "dataset/imdb_top_1000.csv"

def fetch_file_data1(file_url1=FILE_URL1, file_path1=FILE_PATH):
    os.makedirs(file_path1, exist_ok=True)
    csv_path1 = os.path.join(file_path1, "imdb_top_1000.csv")
    urllib.request.urlretrieve(file_url1, csv_path1)

fetch_file_data1()

In [ ]:
#load our data
datapath = os.path.join("dataset","")
imdb = pd.read_csv(datapath + "imdb_top_1000.csv")

In [ ]:
#prova stampa
imdb.head(10)

In [ ]:
#download english stopwards (sinossi dei film in inglese)

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/FiddiCoder/-FIA-Project/main/"
FILE_PATH = os.path.join("dataset")
FILE_URL1 = DOWNLOAD_ROOT + "dataset/stopwords.txt"

def fetch_file_data1(file_url1=FILE_URL1, file_path1=FILE_PATH):
    os.makedirs(file_path1, exist_ok=True)
    csv_path1 = os.path.join(file_path1, "stopwords.txt")
    urllib.request.urlretrieve(file_url1, csv_path1)

fetch_file_data1()

stopwords = open( datapath + "stopwords.txt")
with open(datapath + "stopwords.txt", "r") as f:
    stopwords = f.read().split("\n")

In [ ]:
#creazione lista che contiene le parole principali utili all'algoritmo
def crea_lista(row):
    stars = row[["Star1","Star2","Star3","Star4"]].to_list()
    lista = " ".join(row["Series_Title"].split()*2 + row["Genre"].split()*3 + row["Overview"].split() + row["Director"].split()*2 + stars*2 )
    print(stars)
    print(lista)
    # removing unwanted characters
    lista = re.sub(r'[^\w\s]', '', lista)
        # removing stopwords from the lista
    print(lista)
    wordlist = lista.split()
    for word in wordlist:
        if word in stopwords:
            lista = lista.replace(" "+word+" ", " ")
    
    print(lista)
crea_lista(imdb.loc[0])


['Tim Robbins', 'Morgan Freeman', 'Bob Gunton', 'William Sadler']
The Shawshank Redemption The Shawshank Redemption Drama Drama Drama Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency. Frank Darabont Frank Darabont Tim Robbins Morgan Freeman Bob Gunton William Sadler Tim Robbins Morgan Freeman Bob Gunton William Sadler
The Shawshank Redemption The Shawshank Redemption Drama Drama Drama Two imprisoned men bond over a number of years finding solace and eventual redemption through acts of common decency Frank Darabont Frank Darabont Tim Robbins Morgan Freeman Bob Gunton William Sadler Tim Robbins Morgan Freeman Bob Gunton William Sadler
The Shawshank Redemption The Shawshank Redemption Drama Drama Drama Two imprisoned men bond number years finding solace eventual redemption acts common decency Frank Darabont Frank Darabont Tim Robbins Morgan Freeman Bob Gunton William Sadler Tim Robbins Morgan Freeman Bob Gunton William S